In [1]:
!nvidia-smi

Sat Jun  3 02:01:41 KST 2023

+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:14:00.0 Off |                    0 |
| N/A   32C    P0    24W / 250W |      4MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:15:00.0 Off |                    0 |
| N/A   32

In [2]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch
import nibabel as nib

from monai.apps import download_and_extract
from monai.config import print_config
from monai.utils import set_determinism

print_config()
set_determinism(0)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

MONAI version: 1.1.0
Numpy version: 1.23.5
Pytorch version: 1.7.1+cu110
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a2ec3752f54bfc3b40e7952234fbeb5452ed63e3
MONAI __file__: /home/aistore3/.conda/envs/snuimageseg/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.8
Nibabel version: 5.0.0
scikit-image version: 0.19.3
Pillow version: 9.4.0
Tensorboard version: 2.11.0
gdown version: 4.6.0
TorchVision version: 0.8.2+cu110
tqdm version: 4.64.1
lmdb version: 1.4.0
psutil version: 5.9.0
pandas version: 1.5.2
einops version: 0.6.0
transformers version: 4.25.1
mlflow version: 2.1.1
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [3]:
image_dir = "marked"
uncropped_dir = "nifti"

In [4]:
patients = sorted(glob.glob(os.path.join(uncropped_dir, "*.nii.gz")))
print(len(patients))

246


In [5]:
patient_data = pd.read_excel("new_nifti_center2.xlsx", usecols=['patID', 'v_center', 'h_center', 'l_center']).set_index("patID").dropna().astype('int')
print(patient_data)

       v_center  h_center  l_center
patID                              
1            88       133        60
2           113       185        67
3            81       209        39
7           175       280        78
10          180       259        40
...         ...       ...       ...
348         104       212        52
349         155       176        43
350         190       179        60
351         154       125        53
352         270       326        71

[231 rows x 3 columns]


In [6]:
def nii_loader(filename) :
    import nibabel as nib
    import numpy as np
    import matplotlib.pyplot as plt
    nimg = nib.load(filename)
    return nimg.get_fdata(), nimg.affine, nimg.header

def mark_nifti(patients, idx, patient_data):
    image = patients[idx]
    image_data, image_affine, image_header = nii_loader(image)
    image_shape = image_data.shape
    
    patID = int(os.path.basename(image).split('.')[0])
    center_coords = patient_data.loc[patID, "v_center":"l_center"]
    target_layer = center_coords[2]
    target_image = np.rot90(image_data[:,:,target_layer], 1)
    
    mark_size = image_shape[0]//5
    x = center_coords[0] - mark_size//2
    y = image_shape[1] - center_coords[1] - mark_size//2
    
    fig, ax = plt.subplots()
    ax.imshow(target_image, cmap='gray')
    ax.add_patch(patches.Rectangle((x, y), mark_size, mark_size, edgecolor = 'red', fill=False))
    ax.text(image_shape[0]*0.3, image_shape[1]*0.85, f'[patID] {patID}', color='white')
    ax.text(image_shape[0]*0.3, image_shape[1]*0.9, f'[dimension] {image_shape}', color='white')
    plt.savefig(os.path.join(image_dir, str(patID) + '_layer' + str(center_coords[2]) + '.png'))
    plt.close()

In [7]:
for i in range(len(patients)):
    try:
        mark_nifti(patients, i, patient_data)
    except KeyError:
        print("Failed to process {}".format(patients[i]))

Failed to process nifti/100.nii.gz
Failed to process nifti/162.nii.gz
Failed to process nifti/23.nii.gz
Failed to process nifti/233.nii.gz
Failed to process nifti/286.nii.gz
Failed to process nifti/288.nii.gz
Failed to process nifti/290.nii.gz
Failed to process nifti/309.nii.gz
Failed to process nifti/315.nii.gz
Failed to process nifti/322.nii.gz
Failed to process nifti/329.nii.gz
Failed to process nifti/355.nii.gz
Failed to process nifti/45.nii.gz
Failed to process nifti/76.nii.gz
Failed to process nifti/89.nii.gz
